# Tool de busca de temperatura


In [58]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

In [59]:
from langchain.agents import tool
from langchain.pydantic_v1 import BaseModel, Field

import requests
import datetime

class ReturnTempArgs(BaseModel):
    latitude: float = Field( description='Latitude da localização')
    longitude: float = Field(description='Longitude da localização')

@tool(args_schema=ReturnTempArgs)
def return_current_temperature(latitude:float, longitude:float):
    '''Retorna a temperatura atual de uma localização geográfica'''

    URL = 'https://api.open-meteo.com/v1/forecast'

    params = {
        'latitude':latitude,
        'longitude':longitude,
        'hourly':'temperature_2m',
        'forecast_2m':1
    }

    response = requests.get(URL, params=params)
    response

    if response.status_code == 200:
        result = response.json()
        current_time = datetime.datetime.now(datetime.UTC).replace(tzinfo=None)
        list_hours = [datetime.datetime.fromisoformat(temp_str) for temp_str in result['hourly']['time']]
        closest_index = min(range(len(list_hours)), key=lambda x: abs(list_hours[x] - current_time))
        
        current_temperature = result['hourly']['temperature_2m'][closest_index]
        
        return current_temperature
    else:
        raise Exception('Erro ao acessar a API Open Meteo')
        


In [60]:
print(return_current_temperature)

name='return_current_temperature' description='return_current_temperature(latitude: float, longitude: float) - Retorna a temperatura atual de uma localização geográfica' args_schema=<class '__main__.ReturnTempArgs'> func=<function return_current_temperature at 0x00000219D1882A20>


In [61]:
print(return_current_temperature.args)

{'latitude': {'title': 'Latitude', 'description': 'Latitude da localização', 'type': 'number'}, 'longitude': {'title': 'Longitude', 'description': 'Longitude da localização', 'type': 'number'}}


In [62]:
return_current_temperature.invoke({'latitude': -19.912998, 'longitude':-43.940933})

26.5

# Tool de busca no Wikipedia

In [63]:
import wikipedia
wikipedia.set_lang('pt')

query = 'Isaac Asimov'

@tool
def search_wikipedia(query:str):
    '''Search for a query in Wikipedia and return the summary of the first 3 pages found'''
    titles_pages = wikipedia.search(query)
    summarys = []

    for title in titles_pages[:3]:
        try:
            wiki_page = wikipedia.page(title=title, auto_suggest=True)
            summarys.append(f'Título da Página: {title}\n\n{wiki_page.summary}')
        except:
            pass
        
    if not summarys:
        return 'There are no pages found for this query'
    else:
        return '\n\n'.join(summarys)
        

In [64]:
print(search_wikipedia.invoke({'query':'Isaac Asimov'}))

Título da Página: Isaac Asimov

Isaac Asimov (em russo: Исаак Юдович Озимов; romaniz.: Isaak Yudavich Azimov; Petrovichi, Rússia Soviética, atual Rússia, 2 de janeiro de 1920 — Brooklyn, 6 de abril de 1992) foi um escritor e bioquímico russo-americano, autor de obras de ficção científica e divulgação científica.
Asimov é considerado um dos mestres da ficção científica e, junto com Robert A. Heinlein e Arthur C. Clarke, foi considerado um dos "três grandes" dessa área da literatura. A obra mais famosa de Asimov é a Série da Fundação, também conhecida como Trilogia da Fundação, que faz parte da série do Império Galáctico e que logo combinou com a Série Robôs. Também escreveu obras de mistério e fantasia, assim como uma grande quantidade de não-ficção. No total, escreveu ou editou mais de 500 volumes, aproximadamente 90 000 cartas ou postais, e tem obras em cada categoria importante do sistema de classificação bibliográfica de Dewey, exceto em filosofia.
A maioria de seus livros mais popu

In [65]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.utils.function_calling import convert_to_openai_function

prompt = ChatPromptTemplate.from_messages([
    ('system', 'Você é um assistente amigável chamado Isaac'),
    ('user', '{input}')
])

chat = ChatOpenAI()

tools = [search_wikipedia, return_current_temperature]
tools_json = [convert_to_openai_function(tool) for tool in tools ]
tool_run = {tool.name: tool for tool in tools}

chain = prompt | chat.bind(functions=tools_json)

In [67]:
print(tool_run['search_wikipedia'].invoke({'query':'Isaac Asimov'}))

Título da Página: Isaac Asimov

Isaac Asimov (em russo: Исаак Юдович Озимов; romaniz.: Isaak Yudavich Azimov; Petrovichi, Rússia Soviética, atual Rússia, 2 de janeiro de 1920 — Brooklyn, 6 de abril de 1992) foi um escritor e bioquímico russo-americano, autor de obras de ficção científica e divulgação científica.
Asimov é considerado um dos mestres da ficção científica e, junto com Robert A. Heinlein e Arthur C. Clarke, foi considerado um dos "três grandes" dessa área da literatura. A obra mais famosa de Asimov é a Série da Fundação, também conhecida como Trilogia da Fundação, que faz parte da série do Império Galáctico e que logo combinou com a Série Robôs. Também escreveu obras de mistério e fantasia, assim como uma grande quantidade de não-ficção. No total, escreveu ou editou mais de 500 volumes, aproximadamente 90 000 cartas ou postais, e tem obras em cada categoria importante do sistema de classificação bibliográfica de Dewey, exceto em filosofia.
A maioria de seus livros mais popu

## Roteamento para execução automática de tools

Chamamos de roteamento o processo de análise da output de um modelos que possui acesso a ferramntas. Noste processo, precisamos entender qual é o tipo de saída que o modelo está nos fornecendo. Caso seja uma saída já com a resposta final do modelo, podemos retorná-la ao usuário, caso o modelo esteja solicitando a chamado de uma ferramenta, temos que chamar essa ferramenta e mostrar o resultado ao usuário.

### Adicionando OpenAIFunctionsAgentOutputParser

Para auxiliar neste processo, podemos utilizar o OpenAIFunctionsAgentOutputParser. Ele processa a saída de um modelo da OpenAi que possui ferramentas e determina o estado da mensagem devolvida.

In [68]:
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

chain = prompt | chat.bind(functions=tools_json) | OpenAIFunctionsAgentOutputParser()

O retorno será um AgentAction caso necessite que uma ferramenta seja executada.

In [71]:
result = print(chain.invoke({'input': 'Quem foi Isaac Asimov?'}))

tool='search_wikipedia' tool_input={'query': 'Isaac Asimov'} log="\nInvoking: `search_wikipedia` with `{'query': 'Isaac Asimov'}`\n\n\n" message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"Isaac Asimov"}', 'name': 'search_wikipedia'}}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 142, 'total_tokens': 162, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-186185a1-3245-46b5-ba2f-4b6aff63807e-0')]


### Rodando as ferramentas

Podemos criar uma função simples de roteamento para lidar com os dois estados possíveis:

In [73]:
from langchain.agents.agent import AgentFinish

def routing(result):
    if isinstance(result, AgentFinish):
        return result.return_values['output']
    else:
        return tool_run[result.tool].run(result.tool_input)

In [74]:
chain = prompt | chat.bind(functions=tools_json) | OpenAIFunctionsAgentOutputParser() | routing

In [75]:
print(chain.invoke({'input': 'Quem foi Isaac Asimov?'}))

Título da Página: Isaac Asimov

Isaac Asimov (em russo: Исаак Юдович Озимов; romaniz.: Isaak Yudavich Azimov; Petrovichi, Rússia Soviética, atual Rússia, 2 de janeiro de 1920 — Brooklyn, 6 de abril de 1992) foi um escritor e bioquímico russo-americano, autor de obras de ficção científica e divulgação científica.
Asimov é considerado um dos mestres da ficção científica e, junto com Robert A. Heinlein e Arthur C. Clarke, foi considerado um dos "três grandes" dessa área da literatura. A obra mais famosa de Asimov é a Série da Fundação, também conhecida como Trilogia da Fundação, que faz parte da série do Império Galáctico e que logo combinou com a Série Robôs. Também escreveu obras de mistério e fantasia, assim como uma grande quantidade de não-ficção. No total, escreveu ou editou mais de 500 volumes, aproximadamente 90 000 cartas ou postais, e tem obras em cada categoria importante do sistema de classificação bibliográfica de Dewey, exceto em filosofia.
A maioria de seus livros mais popu